# Wallet API v2 - App Walkthtough

## Table of Content
* [Overview](#overview)
* [Prerequisites](#prereq)
    * [Required dependencies](#req-dep)
    * [Create keys](#create-keys)
    * [Project setup](#proj-setup)
    * [Load API credentials](#load-api)
    * [Instantiate CDP client](#cdp-client)
* [1. Create EOA Wallet](#create-eoa)
    * [1.1 wallet.py](#eoa-wallet)
    * [1.2 app.py](#eoa-main)
* [2. Create Smart Contract Wallet](#create-smart)
    * [2.1 wallet.py](#smart-wallet)
    * [2.2 app.py](#smart-main)
* [3. Fund Smart Wallet with Sepolia ETH](#faucet)
    * [3.1 wallet.py](#faucet-wallet)
    * [3.2 app.py](#faucet-main)
* [4. Transfer Sepolia ETH from Smart Contract wallet to a different wallet](#transfer)
    * [4.1 transaction.py](#transfer-transaction)
    * [4.2 app.py](#transfer-main)

## Overview <a class="anchor" id="overview"></a>
The [Wallet API v2](https://docs.cdp.coinbase.com/wallet-api-v2/docs/welcome) allows you to create account and perform operations on EVM compatible networks.

In this example, you will utilize Wallet API v2 through [Python CDP SDK](https://coinbase.github.io/cdp-sdk/python/), a library that provides a client for interacting with the [Coinbase Developer Platform (CDP)](https://docs.cdp.coinbase.com/), to build a simple [CLI-based app](https://github.com/michelleflion/cb-test) with multiple functionalities on EVM (Ethereum Virtual Machine) .

Through this example, you will learn how to:
1. Generate EOA (Externally Owned Accounts) on EVM
2. Generate Smart Contract wallets on EVM
3. Fund the Smart Contract wallet created with Sepolia Ethereum (ETH) using the Coinbase faucet
4. Transfer the ETH funded above to a different EVM wallet of choice

## Prerequisites <a class="anchor" id="prereq"></a>
Setup all dependencies, export your keys to environment variables, and initialize a new project before you begin.


### Required dependencies <a class="anchor" id="req-dep"></a>
For this example, ensure that you have:
* Python 3.10+ installed
    * You may download [here](https://www.python.org/downloads/) or use any distribution of your choice
* `cdp-sdk`, `python-dotenv`, `web3` packages installed from `pip`
    * See instructions below on <i>[Project setup](#proj-setup)</i> if you do not already have these packages in your environment
* Created and signed in to an existing [CDP account]((https://portal.cdp.coinbase.com/))
  
Once you have setup the prerequisite dependencies, proceed below to create keys to authenticate your requests and initialize a new project.

### Create keys <a class="anchor" id="create-keys"></a>
Sign in to the [CDP Portal](https://portal.cdp.coinbase.com/), [create a CDP API key](https://portal.cdp.coinbase.com/projects/api-keys) and [generate a Wallet Secret](https://portal.cdp.coinbase.com/products/wallet-api). Keep these values handy as you will need them in the following steps.

For more information, see the [CDP API Keys](https://docs.cdp.coinbase.com/get-started/docs/cdp-api-keys#secret-api-keys) and [Wallet Secret](https://docs.cdp.coinbase.com/wallet-api-v2/docs/security#wallet-secret) documentation.

Ensure to copy and store these values somewhere safe for the next section on <i>[Load API credentials]((#load-api))</i>.

### Project setup <a class="anchor" id="proj-setup"></a>
After creating your keys, you will need to initialize a new project and instantiate the CDP client.

Initialize a new Python project by running the command below on the Terminal of your choice (below commands are based on `Zsh`):

```bash
mkdir cdp-python-example && cd cdp-python-example && python -m venv .venv && source .venv/bin/activate && touch app.py && touch .env && mkdir utils && touch utils/wallet.py && touch utils/transaction.py
```

The above command will:
1. Create a new project root folder in your current directory for the app `cdp-python-example`
2. Navigate into the new folder created
3. Create a new Python virtual environment `.venv` to prevent any dependency version conflicts
4. Activate the virtual environment created above
    1. Note that the activate path may vary slightly depending on your terminal and OS used
    2. You should see `(.venv)` at the start of the command line once the virtual environment has been activated successfully
7. Create 2 new files in the new project folder
    1. `app.py` for the main project script to run
    2. `.env` for storing API credentials
8. Create a new folder `utils` to store app functions
    1. Create 2 new files `wallet.py` and `transaction.py` to store the helper functions
   

Once you are in the virtual environment, you may install the package dependencies accordingly by running either one of the below:

Install each package libraries manually by running the following on your Terminal:
```bash
pip install cdp-sdk python-dotenv web3
```

Alternatively, install all required packages by running the following if you have downloaded `requirements.txt` from the Github Repo:
```bash
pip install -r requirements.txt
```


### Load API credentials <a class="anchor" id="load-api"></a>
Add the folowing lines into the `.env` file created above using the key values created in the <i>[Create keys](#create-keys)</i> section.

Replace the variable values with your respective CDP account secrets.

file: `./.env`
```python 
CDP_API_KEY_ID=your-api-key-id
CDP_API_KEY_SECRET=your-api-key-secret
CDP_WALLET_SECRET=your-wallet-secret
```

These variables will be passed to the Client and loaded accordingly.

### Instantiate CDP client <a class="anchor" id="cdp-client"></a>

You are now ready to connect to the CDP client instance

file: `./app.py`

```python
import logging
from cdp import CdpClient
import asyncio
from dotenv import load_dotenv

load_dotenv() # This line will load your API key and secret variables from .env

async def main():
    # Initialize the CDP client, which automatically loads the API Key and 
    # Wallet Secret from the environment variables.
    cdp = CdpClient()
    logging.info(f"Successfully connected to CDP client")
    await cdp.close()


asyncio.run(main())
```

Once saved, you may run the code on your Terminal from your project directory:

```bash
>> python app.py

Successfully connected to CDP client
```

Now that you have successfully instantiate CDP Client on your app, you are ready to build the different app functions under `utils`.

## CLI App structure <a class="anchor" id="app-structure"></a>
You will be using `app.py` as the main script file as a parser to the CLI.

Start with the following code to replace previous code used for instantiation above:

file: `./app.py`
```python
import argparse

from dotenv import load_dotenv


# Load .env variables
load_dotenv()

# Main app function
def main():

    # Initialize CLI Argument Parsing
    parser = argparse.ArgumentParser(description="Coinbase Wallet API v2 App")

    parser.print_help()

# Run main() function
if __name__ == '__main__':
    main()
```

You should see the following help message printed out now for the created parser
```bash
>> python app.py

usage: app.py [-h]

Coinbase Wallet API v2 App

options:
  -h, --help  show this help message and exit
```

## Logging <a class="anchor" id="logging"></a>
Before diving into the app, configure logging for good practice used across Python libraries.


file: `./app.py`
```python
import logging
import argparse

from dotenv import load_dotenv


# Load .env variables
load_dotenv()

# Configure logging
def configure_logging(level):
    logging.basicConfig(
        level=level,
        format="%(asctime)s - %(name)s - %(levelname)s - %(message)s",
    )

# Main app function
def main():

    # Initialize CLI Argument Parsing
    parser = argparse.ArgumentParser(description="Coinbase Wallet API v2 App")

    # Add an argument for app logging level
    parser.add_argument(
        '--log-level',
        choices=['DEBUG', 'INFO', 'WARNING', 'ERROR', 'CRITICAL'],
        default='INFO',
        help="Set the logging level (default: INFO)"
    )

    # Parse args provided
    args = parser.parse_args()

    # Configure logging based on the user’s chosen level
    log_level = getattr(logging, args.log_level)
    configure_logging(log_level)

    logging.info(f"Logging configured to {args.log_level}")

    parser.print_help()

# Run main() function
if __name__ == '__main__':
    main()
```

You should see the following log printed out on your terminal now when you run the app:
```bash
>> python app.py

<timestamp> - root - INFO - Logging configured to INFO
usage: app.py [-h] [--log-level {DEBUG,INFO,WARNING,ERROR,CRITICAL}]

Coinbase Wallet API v2 App

options:
  -h, --help            show this help message and exit
  --log-level {DEBUG,INFO,WARNING,ERROR,CRITICAL}
                        Set the logging level (default: INFO)
```

## 1. Create EOA Wallet <a class="anchor" id="create-eoa"></a>
EOA (Externally Owned Accounts) are accounts on any EVM-compatible blockchain that have the ability to sign transactions on behalf of an account's address (i.e., when using a smart account). Read more [here](https://docs.cdp.coinbase.com/wallet-api-v2/docs/accounts) for the different types of accounts supported.

This function will create an EOA with an optional `name` parameter to be associated with the newly created address.

### 1.1 wallet.py <a class="anchor" id="eoa-wallet"></a>
Head to `utils` and add the following code block into the empty `wallet.py` file:


file: `./utils/wallet.py`
```python
import logging
import asyncio

from cdp import CdpClient

from dotenv import load_dotenv

load_dotenv()


async def create_eoa_wallet(name=None):
    logging.debug('Running create_eoa_wallet')
    try:
        async with CdpClient() as cdp: # Instantiate CDP Client 
            account = await cdp.evm.create_account(name)
            logging.info(f"Created EVM account: {account.address} with name {name}")
        return account
    except Exception as e:
        logging.exception('Exception error:')
```

### 1.2 app.py <a class="anchor" id="eoa-main"></a>
Add this newly created function as a subcommand in your CLI parser under the `main()` function

file: `./app.py`

Import the `asyncio` package and `wallet` module into `app.py`
```python
import asyncio

# Importing created functions from utils directory
from utils.wallet import create_eoa_wallet
```

Add the `command` subparser for `create-eoa` in `main()` function

```python
# Main app function
def main():

    # Initialize CLI Argument Parsing
    parser = argparse.ArgumentParser(description="Coinbase Wallet API v2 App")

    # Add an argument for app logging level
    parser.add_argument(
        '--log-level',
        choices=['DEBUG', 'INFO', 'WARNING', 'ERROR', 'CRITICAL'],
        default='INFO',
        help="Set the logging level (default: INFO)"
    )

    # Adding a command subparser for the different functions
    subparsers = parser.add_subparsers(dest="command")

    # Subcommand for creating EOA wallet
    eoa_parser = subparsers.add_parser("create-eoa", help="Create an Externally Owned Account (EOA) wallet")
    eoa_parser.add_argument("--name", type=str, default=None, help="Name assigned to the account created for future reference (Default: None)")

    # Parsing args provided
    args = parser.parse_args()

    # Configure logging based on the user’s chosen level
    log_level = getattr(logging, args.log_level)
    configure_logging(log_level)

    logging.info(f"Logging configured to {args.log_level}")

    # Running respective functions
    if args.command == "create-eoa":
        asyncio.run(create_eoa_wallet(args.name))
    else:
        parser.print_help()


# Run main() function
if __name__ == '__main__':
    main()
```

You can now head to Terminal and run the command:

```bash
>> python app.py create-eoa --name cdp-test

<timestamp> - root - INFO - Logging configured to INFO
<timestamp> - root - INFO - Created EVM account: 0x7E... with name cdp-test


## 2. Create Smart Contract Wallet<a class="anchor" id="create-smart"></a>
[Smart accounts](https://docs.cdp.coinbase.com/wallet-api-v2/docs/accounts#smart-accounts) are a type of account that can be used to execute user operations onchain.

This function will create a smart account with an optional specified owner_address parameter to sign on the smart account. If not provided, it will create a new EOA to be assigned as an owner.

### 2.1 wallet.py <a class="anchor" id="smart-wallet"></a>
Head to `utils` and add the following code block into the `wallet.py` file:


file: `./utils/wallet.py`
```python
async def create_smart_wallet(owner_address=None):
    logging.debug('Running create_smart_wallet')
    try:
        async with CdpClient() as cdp: # Instantiate CDP Client
            if owner_address is None: # If owner_address is not provided
                account = await create_eoa_wallet()
                logging.info(f"Owner account created {account.address}")
            else:
                account = await cdp.evm.get_account(address=owner_address)
                logging.info(f"Owner account obtained {account.address}")
            
            smart_account = await cdp.evm.create_smart_account(account)
            logging.info(f"Created Smart account: {smart_account.address} with owner {smart_account.owners}")
        return smart_account
    except Exception as e:
        logging.exception('Exception error:')
```

### 2.2 app.py <a class="anchor" id="smart-main"></a>
Add this newly created `create_smart_wallet` function as a second subcommand in your CLI parser under the `main()` function

file: `./app.py`

Import the new function from the `wallet` module into `app.py`
```python
from utils.wallet import create_eoa_wallet, create_smart_wallet
```

Add the subparser for `create-smart` in `main()` function below the one for `create-eoa`

```python
    # Subcommand for creating a Smart Contract wallet
    smart_parser = subparsers.add_parser("create-smart", help="Create a Smart Contract wallet")
    smart_parser.add_argument("--owner-address", type=str, default=None, help="Address of owner wallet if previously created (Default: None)")
```

Call the function based on CLI command
```python
    # Running respective functions
    if args.command == "create-eoa":
        asyncio.run(create_eoa_wallet(args.name))
    elif args.command == "create-smart":
        asyncio.run(create_smart_wallet(args.owner_address))
    else:
        parser.print_help()
```

You can now head to Terminal and run the command:

```bash
>> python app.py create-smart --owner-address 0x7E...

<timestamp> - root - INFO - Logging configured to INFO
<timestamp> - root - INFO - Owner account obtained 0x7E...
<timestamp> - root - INFO - Created Smart account: 0x22... with owner [Ethereum Account Address: 0x7E...]


## 3. Fund Smart Wallet with Sepolia ETH <a class="anchor" id="faucet"></a>
A faucet is a service that dispenses small amounts of cryptocurrency (testnet tokens) to developers for testing purposes.

This function will fund a test EVM token of choice (default: Sepolia ETH) on the a chosen EVM network (default: [Base Sepolia network](https://docs.base.org/)) in a given address for testing purposes. If no address is provided, it will create a new Smart Contract address.

Refer [here](https://docs.cdp.coinbase.com/faucets/docs/welcome) for more information on supported tokens and limits.

### 3.1 wallet.py <a class="anchor" id="faucet-wallet"></a>
Head to `utils` and add the following code block into the `wallet.py` file:


file: `./utils/wallet.py`
```python
async def fund_wallet(fund_address=None, network="base-sepolia", token="eth"):
    logging.debug('Running fund_wallet')
    try:
        async with CdpClient() as cdp: # Instantiate CDP Client
            if fund_address is None: # If fund_address is not provided
                fund_account = await create_smart_wallet()
                fund_address = fund_account.address
                logging.info(f"Creating new EOA account {fund_address}")

            faucet_hash = await cdp.evm.request_faucet(
                address=fund_address,
                network=network,
                token=token
            )

            if network == "base-sepolia": # Choosing scanner for hash URL
                scanner = "basescan.org"
            else:
                scanner = "etherscan.io"

            logging.info(f"Requested funds from {token} faucet on {network}: https://sepolia.{scanner}/tx/{faucet_hash}")
    except Exception as e:
        logging.exception('Exception error:')
```

### 3.2 app.py <a class="anchor" id="faucet-main"></a>
Add this newly created `fund_wallet` function as a third subcommand in your CLI parser under the `main()` function


file: `./app.py`

Import the new function from the `wallet` module into `app.py`
```python
from utils.wallet import create_eoa_wallet, create_smart_wallet, fund_wallet
```

Add the subparser for `fund-wallet` in `main()` function below the one for `create-eoa`

```python
    # Subcommand for funding a wallet from faucet
    fund_parser = subparsers.add_parser("fund-wallet", help="Fund a wallet with a specified token on a specified network")
    fund_parser.add_argument("--address", type=str, default=None, help="The wallet address for faucet funding. If not set, a new EOA wallet will be created (Default: None)")
    fund_parser.add_argument("--network", type=str, default="base-sepolia", help="The network for faucet funding (Default: base-sepolia)")
    fund_parser.add_argument("--token", type=str, default="eth", help="The token for faucet funding (Default: eth)")
```

Call the function based on CLI command
```python
    # Running respective functions
    if args.command == "create-eoa":
        asyncio.run(create_eoa_wallet(args.name))
    elif args.command == "create-smart":
        asyncio.run(create_smart_wallet(args.owner_address))
    elif args.command == "fund-wallet":
        asyncio.run(fund_wallet(args.address, args.network, args.token))
    else:
        parser.print_help()
```

You can now head to Terminal and run the command:

```bash
>> python app.py fund-wallet --address 0x22...
<timestamp> - root - INFO - Logging configured to INFO
<timestamp> - root - INFO - Requested funds from eth faucet on base-sepolia: https://sepolia.basescan.org/tx/0xf1...
```

If no address is specified:

```bash
>> python app.py fund-wallet                                                     
<timestamp> - root - INFO - Logging configured to INFO
<timestamp> - root - INFO - Created EVM account: 0x23... with name None
<timestamp> - root - INFO - Owner account created 0x23...
<timestamp> - root - INFO - Created Smart account: 0xCD... with owner [Ethereum Account Address: 0x23...]
<timestamp> - root - INFO - Creating new EOA account 0xCD...
<timestamp> - root - INFO - Requested funds from eth faucet on base-sepolia: https://sepolia.basescan.org/tx/0x6c...
```

## 4. Transfer Sepolia ETH from Smart Contract wallet to a different wallet <a class="anchor" id="transfer"></a>

Funds can be transferred from a Smart Contract wallet to a different wallet through [User Operations](https://docs.cdp.coinbase.com/wallet-api-v2/docs/smart-accounts#2-send-a-user-operation).

In this example, we will be transferring Sepolia ETH on the [Base Sepolia network](https://docs.base.org/) from the smart wallet created previously.

This function will transfer a specified amount of Base Sepolia ETH (default: 0) from a smart wallet of choice, to a different wallet of choice.

Refer [here](https://docs.cdp.coinbase.com/wallet-api-v2/docs/smart-accounts#2-send-a-user-operation) for more information on smart wallet operations.

### 4.1 transaction.py <a class="anchor" id="transfer-transaction"></a>
We will be using the `transaction.py` file for transaction-related functions.

Head to `utils` and add the following code block into the new `transaction.py` file:


file: `./utils/transaction.py`
```python
import logging
import asyncio

from cdp import CdpClient
from cdp.evm_call_types import EncodedCall

from dotenv import load_dotenv

from web3 import Web3
from decimal import Decimal


load_dotenv()


async def transfer_baseeth(from_smart_address, from_smart_owner, to_address, amount=0):
    logging.debug('Running transfer_eth')
    try:
        async with CdpClient() as cdp: # Instantiate CDP Client
            # Extract smart account inputs
            owner_account = await cdp.evm.get_account(from_smart_owner)
            smart_account = await cdp.evm.get_smart_account(from_smart_address, owner_account)
            logging.info(f"Obtained {smart_account} with owner {smart_account.owners}")
           
           # Send user operation
            user_operation = await cdp.evm.send_user_operation(
                smart_account= smart_account,
                network="base-sepolia",
                calls=[
                    EncodedCall(
                        to=to_address,
                        data="0x",
                        value=Web3.to_wei(Decimal(amount), "ether"),
                    )
                ],
            )
            logging.info(f"User operation status: {user_operation.status}")

            # Wait for user operation confirmation
            logging.debug("Waiting for user operation to be confirmed...")
            user_operation = await cdp.evm.wait_for_user_operation(
                smart_account_address=smart_account.address,
                user_op_hash=user_operation.user_op_hash,
            )
            
            if user_operation.status == "complete":
                logging.info(f"User operation confirmed. Transaction hash: {user_operation.transaction_hash}")
            else:
                logging.error("User operation failed")
    except Exception as e:
        logging.exception('Exception error:')
```

### 4.2 app.py <a class="anchor" id="transfer-main"></a>
Add this newly created `transfer_baseeth` function as a third subcommand in your CLI parser under the `main()` function

file: `./app.py`

Import the new function from the `wallet` module into `app.py`
```python
from utils.transaction import transfer_baseeth
```

Add the subparser for `fund-wallet` in `main()` function below the one for `create-eoa`

```python
    # Subcommand for transferring ETH from Smart Contract wallet
    transfer_parser = subparsers.add_parser("transfer-baseeth", help="Transfer token from smart wallet to a specified EOA wallet")
    transfer_parser.add_argument("--from-smart-address", type=str, help="The smart wallet address to transfer token from")
    transfer_parser.add_argument("--from-smart-owner", type=str, help="The smart wallet address' owner to sign transactions")
    transfer_parser.add_argument("--to-address", type=str, help="Destination address for transfer")
    transfer_parser.add_argument("--amount", type=str, default="0", help="ETH amount to transfer (Default: 0)")
```

Call the function based on CLI command
```python
    # Running respective functions
    if args.command == "create-eoa":
        asyncio.run(create_eoa_wallet(args.name))
    elif args.command == "create-smart":
        asyncio.run(create_smart_wallet(args.owner_address))
    elif args.command == "fund-wallet":
        asyncio.run(fund_wallet(args.address, args.network, args.token))
    elif args.command == "transfer-baseeth":
        # Check if required arguments are provided
        if not args.from_smart_address: 
            print("Error: --from-smart-address is required for transfer")
            return
        if not args.from_smart_owner:
            print("Error: --from-smart-owner is required for transfer")
            return
        if not args.to_address:
            print("Error: --to-address is required for transfer")
            return
        asyncio.run(transfer_baseeth(args.from_smart_address, args.from_smart_owner, args.to_address, args.amount))
    else:
        parser.print_help()
```

You can now head to Terminal and run the command:

```bash
>> python app.py transfer-baseeth --from-smart-address 0x22... --from-smart-owner 0xA9... --to-address 0xC8...
<timestamp> - root - INFO - Logging configured to INFO
<timestamp> - root - INFO - Obtained Smart Account Address: 0x22... with owner [Ethereum Account Address: 0xA9...]
<timestamp> - root - INFO - User operation status: broadcast
<timestamp> - root - INFO - User operation confirmed. Transaction hash: 0x55...
```

## Congratulations!
You now have a mini CLI app with basic smart account functionalities built on the CDP Python SDK.